In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train = pd.read_csv('data/X_train_encoded_transformed_scaled.csv')
X_test = pd.read_csv('data/X_test_encoded_transformed_scaled.csv')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
import xgboost as xgb
from xgboost import XGBClassifier